# Reproducing Vitaly Feldman MNIST Results

The goal of this notebook is to reproduce the results in https://arxiv.org/abs/2008.11193 which are based on the MNIST model from https://github.com/pytorch/opacus/blob/master/examples/mnist.py

In [1]:
import numpy as np
import syft as sy
from syft.core.tensor.tensor import Tensor
from syft.core.adp.entity import Entity

In [2]:
data_batch = np.random.rand(4,28*28)
label_batch = np.random.rand(4,10)  

bob = Entity(unique_name="Bob")

data = Tensor(data_batch).autograd(requires_grad=True)#.private(0.01,1,entity=bob)
target = Tensor(label_batch).autograd(requires_grad=True)#.private(0.01,1,entity=bob)

In [3]:
weights = np.random.rand(28*28,10)

pred = data.dot(weights)
diff = target - pred
pre_loss = np.square(diff)
loss = pre_loss.sum() / float(np.prod(pre_loss.shape))
# extraneous_thing = -diff

(AutogradTensor(child=[[0.73969333 0.52077244 0.44865719 ... 0.99420724 0.50380855 0.88681564]
 [0.87159794 0.68644254 0.9298797  ... 0.92710859 0.06512471 0.7182572 ]
 [0.51634192 0.58028294 0.44032024 ... 0.26887025 0.05209138 0.57570296]
 [0.34655305 0.70400183 0.97997124 ... 0.89310128 0.82516314 0.46348291]]), 10)
{}
10
None
(AutogradTensor(child=[0.73969333 0.73969333 0.73969333 ... 0.46348291 0.46348291 0.46348291]), 4, 784, 10)
{}
(AutogradTensor(child=[[[0.73969333 0.73969333 0.73969333 ... 0.73969333 0.73969333 0.73969333]
  [0.52077244 0.52077244 0.52077244 ... 0.52077244 0.52077244 0.52077244]
  [0.44865719 0.44865719 0.44865719 ... 0.44865719 0.44865719 0.44865719]
  ...
  [0.99420724 0.99420724 0.99420724 ... 0.99420724 0.99420724 0.99420724]
  [0.50380855 0.50380855 0.50380855 ... 0.50380855 0.50380855 0.50380855]
  [0.88681564 0.88681564 0.88681564 ... 0.88681564 0.88681564 0.88681564]]

 [[0.87159794 0.87159794 0.87159794 ... 0.87159794 0.87159794 0.87159794]
  [0.6864

In [4]:
loss.backward()
pred.grad

called backward on:<syft.core.tensor.autograd.ops.mul.MulOp object at 0x7f92037ae550> Counter({UUID('c99916ed-70f1-4845-afbd-e522380059c8'): 1}) []
1 0
Adding grad:<class 'syft.core.tensor.autograd.tensor.AutogradTensor'> w/ backprop_id:c99916ed-70f1-4845-afbd-e522380059c8
called backward on:<syft.core.tensor.autograd.ops.sum.SumOp object at 0x7f92037a38b0> Counter({UUID('c99916ed-70f1-4845-afbd-e522380059c8'): 1}) [<syft.core.tensor.autograd.ops.mul.MulOp object at 0x7f92037ae550>]
1 1
40
AutogradTensor(child=[0.025])
(AutogradTensor(child=[0.025]), 40, 'axis')
{}
40
axis


TypeError: 'str' object cannot be interpreted as an integer

In [12]:
import torch as th
_data = th.tensor(data.child.child, requires_grad=True)
_target = th.tensor(target.child.child, requires_grad=True)
_weights = th.tensor(weights, requires_grad=True)

_pred = th.tensor(_data.mm(_weights), requires_grad=True)
_diff = _target - _pred
_pre_loss = _diff*_diff
_loss = _pre_loss.sum(axis=1) / _pre_loss.shape[1]

_loss.sum().backward()
_pred.grad

<ipython-input-12-c60d89d1a38c>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  _pred = th.tensor(_data.mm(_weights), requires_grad=True)


tensor([[40.9471, 38.5175, 40.0879, 39.8305, 39.8029, 38.1938, 40.4757, 40.1643,
         40.5607, 41.7374],
        [38.7393, 37.2251, 38.6177, 36.9597, 38.2239, 37.2290, 39.3364, 39.4739,
         38.9341, 39.3412],
        [40.3075, 39.1607, 40.2972, 39.7583, 39.9539, 39.0881, 40.9623, 41.7433,
         41.5841, 41.1269],
        [39.9529, 38.6192, 40.6448, 38.2046, 39.4253, 38.4149, 39.5353, 41.1847,
         40.0971, 40.4371]], dtype=torch.float64)

In [78]:
self = _data
other = _weights

# _reshaped_self = self.unsqueeze(2).expand(4,784,10)

# _expanded_other = other.unsqueeze(2).expand(784,10,4).transpose(1,2).transpose(0,1)

# _prod = _reshaped_self * _expanded_other

# _result = _prod.sum(1)
_result = self.mm(other)

In [79]:
_result.backward(_result*0+1)

In [80]:
_data.grad

tensor([[4.4891, 4.3078, 4.8468,  ..., 5.8105, 3.5475, 3.9656],
        [4.4891, 4.3078, 4.8468,  ..., 5.8105, 3.5475, 3.9656],
        [4.4891, 4.3078, 4.8468,  ..., 5.8105, 3.5475, 3.9656],
        [4.4891, 4.3078, 4.8468,  ..., 5.8105, 3.5475, 3.9656]],
       dtype=torch.float64)

In [ ]:
diff = pred-target

In [6]:
diff.requires_grad

True

In [7]:
pre_loss = np.square(diff)

In [8]:
pre_loss.requires_grad

True

In [ ]:
# self = data
# other = weights

# repeated_self = self.repeat(other.shape[1])
# reshaped_self = repeated_self.reshape(self.shape[0], self.shape[1], other.shape[1])

# expanded_other = other.repeat(self.shape[0]).reshape(other.shape[0], other.shape[1], self.shape[0]).transpose(2,0,1)

# prod = reshaped_self * expanded_other
# result = prod.sum(1)

In [9]:
# loss = np.mean(pre_loss, axis=1) 

In [10]:
pre_loss.backward()

Adding grad:<class 'autograd.AutogradTensor'>
Adding grad:<class 'autograd.AutogradTensor'>
Adding grad:<class 'autograd.AutogradTensor'>
Adding grad:<class 'autograd.AutogradTensor'>
Adding grad:<class 'autograd.AutogradTensor'>
Adding grad:<class 'autograd.AutogradTensor'>


RecursionError: maximum recursion depth exceeded

In [22]:
import os

os.environ['KMP_DUPLICATE_LIB_OK']='True'

import numpy as np

from syft.lib.adp.scalar import PhiScalar
from syft.lib.adp.entity import Entity
from syft.lib.adp.adversarial_accountant import AdversarialAccountant
from syft.lib.adp.publish import publish

from torchvision import datasets, transforms
import torch
import torch as th

MNIST_MEAN = 0.1307
MNIST_STD = 0.3081
generator = None
sample_rate = 0.001

train_dataset = datasets.MNIST(
    './',
    train=True,
    download=True,
    transform=transforms.Compose(
        [
            transforms.ToTensor(),
            transforms.Normalize((MNIST_MEAN,), (MNIST_STD,)),
        ]
    ),
)

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    generator=generator,
#     **kwargs,
)

batch_size = 4

data_batch = torch.cat([train_dataset[i][0] for i in range(batch_size)]).reshape(-1,28*28) * 1
label_batch = [train_dataset[i][1] for i in range(batch_size)].copy()



In [15]:
from cocos import numerics as cn
import numpy as np
from passthrough import PassthroughTensor
from passthrough import implements

class SymEngineTensor(PassthroughTensor):
    """It turns out that when you're using symengine polynomials 
    there's a non-linear slowdown as you sum across larger and
    larger vectors. Subclassing from this class ensures that sums
    across large vectors are batched for faster performance"""
    
    
    def sum(self, dim, **kwargs):
        
        # when summing across symengine symbols vector sizes of 32 
        # seem to be optimal in terms of performance. there's a strange
        # non-linear increase in performance time when going higher
        batch_size = 512
        overall_shape = self.shape[dim]
        n_batches = int(overall_shape/batch_size)

        results = list()
        end = 0
        for i in range(n_batches-1):
            start = i*batch_size
            end = ((i+1)*batch_size)
            results.append(self.child[:,start:end,:].sum(dim))

        results.append(self.child[:,end:,:].sum(dim))
        out2 = sum(results)
        return SymEngineTensor(out2)
    
    def dot(self, other):
    
        expanded_self = self.child.repeat(other.shape[1]).reshape(self.shape[0], self.shape[1], other.shape[1])
        expanded_other = other.child.repeat(self.shape[0]).reshape(other.shape[0], other.shape[1], self.shape[0]).transpose(2,0,1)
        
        return SymEngineTensor(expanded_other * expanded_self).sum(1)



from syft.lib.adp.tensor import make_entities

data = Tensor(data_batch.numpy())
target = Tensor(np.eye(10)[label_batch])
entities = make_entities(n=len(data))
data = data.private(min_val=-0.42421296, max_val=2.8214867, entities=entities)
target = target.private(min_val=0, max_val=1, entities=entities)
data = data.reshape(-1,28*28)
weights = np.random.rand(28 * 28, 10)

data = SymEngineTensor(data)
weights = SymEngineTensor(weights)
# d

AttributeError: 'numpy.ndarray' object has no attribute 'numpy'

In [1]:
import pymbolic as pmbl
import numpy as np

vs = list()
for i in range(2*28*28):
    x = pmbl.var("s"+str(i))
    vs.append(x)
    
ws = list()
for i in range(10*28*28):
    w = pmbl.var("w"+str(i))
    ws.append(x)    
    
data = np.array(vs).reshape(2,28*28)
weights = np.array(ws).reshape(28*28,10)

In [2]:
%%timeit
pred = data.dot(weights)

58.3 ms ± 270 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [3]:
from syft.lib.adp.tensor import Tensor
from syft.lib.adp.tensor import make_entities


data = Tensor(np.random.rand(2,28*28))
target = Tensor(np.random.rand(28*28,10))
entities = make_entities(n=len(data))
data = data.private(min_val=-0.42421296, max_val=2.8214867, entities=entities)
data = data.reshape(2,28*28)
weights = np.random.rand(28 * 28, 10)

making entities


In [4]:
%%timeit -n1 -r1
out = data.dot(weights)

467 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [1]:
import cocos.device as cd
from cocos.symbolic import (
    LambdifiedMatrixExpression, \
    find_length_of_state_vectors
)
cd.info()
from cocos import numerics as cn

Cocos running on ArrayFire v3.5.1 (OpenCL 64bit)
[0] Apple: AMD Radeon Pro Vega 56 Compute Engine | OpenCL | compute version 1.2


In [14]:
data = cn.random.rand(500000,28*28)
weights = cn.random.rand(28*28,10)

In [15]:
%%timeit -n1 -r1
out = data.dot(weights)

297 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


##### out

In [54]:
out2 = np.dot(data,weights)

In [62]:
sp = out2[0][0].value.poly

In [ ]:
class ScalarTensor(np.ndarray):
    def __new__(
        cls,
        data
    ):
        obj = np.asarray(data).view(cls)
        return obj
    
    def __array_finalize__(self, obj):
        print("finalizing")
        if obj is None:
            return

    def __array_wrap__(self, out_arr, context=None):
        
        print(context)
        
        # out_arr.view calls __array_finalize__
        output = out_arr.view(self.__class__)
        
        return output

In [4]:
import cocos.device as cd
from cocos.symbolic import (
    LambdifiedMatrixExpression, \
    find_length_of_state_vectors
)
cd.info()

Cocos running on ArrayFire v3.5.1 (OpenCL 64bit)
[0] Apple: AMD Radeon Pro Vega 56 Compute Engine | OpenCL | compute version 1.2


In [5]:
import sympy as sym
import numpy as np

In [8]:
# x1, x2, x3, t = sym.symbols('x1, x2, x3, t')
# argument_symbols = (x1, x2, x3)
# g = sym.Function('g')
# f = sym.Matrix([[x1 + x2], [(g(t) * x1 + x3) ** 2], [sym.exp(x1 + x2 + g(t))]])
# jacobian_f = f.jacobian([x1, x2, x3])

# def numeric_time_function(t: float):
#     return np.log(t)

# jacobian_f_lambdified \
#     = LambdifiedMatrixExpression(
#         argument_symbols=argument_symbols,
#         time_symbol=t,
#         symbolic_matrix_expression=jacobian_f,
#         symbolic_time_function_name_to_numeric_time_function_map={'g': numeric_time_function})

# n = 10000000
# X_gpu = cn.random.rand(n, 3)
# X_cpu = np.array(X_gpu)

In [9]:
# %%timeit
# result = X_gpu.sum()

In [10]:
# %%timeit
# result2 = X_cpu.sum()

In [37]:
single_poly = np.array([1,0,1])

single_poly.shape

(3,)

In [14]:
n_polys = (np.random.rand(10,3) > 0.5).astype(int)
n_polys

array([[1, 0, 0],
       [0, 1, 1],
       [0, 1, 1],
       [0, 1, 1],
       [0, 1, 1],
       [0, 1, 0],
       [0, 0, 1],
       [0, 1, 1],
       [0, 0, 0],
       [1, 1, 0]])

In [12]:
x = data.flatten()[0:5]

In [1]:
import autograd.numpy as np   # Thinly-wrapped version of Numpy
from autograd import grad

def taylor_sine(x):  # Taylor approximation to sine function
    ans = currterm = x
    i = 0
    while np.abs(currterm) > 0.001:
        currterm = -currterm * x**2 / ((2 * i + 3) * (2 * i + 2))
        ans = ans + currterm
        i += 1
    return ans

grad_sine = grad(taylor_sine)
print("Gradient of sin(pi) is", grad_sine(x))

NameError: name 'x' is not defined

In [8]:
# %%timeit
# jacobian_f_numeric_gpu = \
#     (jacobian_f_lambdified
#      .evaluate_with_kwargs(x1=X_gpu[:, 0],
#                            x2=X_gpu[:, 1],
#                            x3=X_gpu[:, 2],
#                            t=1.0))

In [9]:
# %%timeit
# jacobian_f_numeric_cpu = \
#     (jacobian_f_lambdified
#      .evaluate_with_kwargs(x1=X_cpu[:, 0],
#                            x2=X_cpu[:, 1],
#                            x3=X_cpu[:, 2],
#                            t=1.0))

In [21]:
print(f'numerical results from cpu and gpu match: '
      f'{np.allclose(jacobian_f_numeric_gpu, jacobian_f_numeric_cpu)}')

numerical results from cpu and gpu match: True


In [7]:
import symengine
from symengine import var

In [8]:
import uuid

In [9]:
syms = ""
for i in range(784):
    id = str(uuid.uuid4()).replace("-","")
    syms += 's'+id+str(i)+' '

In [10]:
v = var(syms)

In [11]:
%%timeit -n1 -r1
result = np.sum(v)

63.8 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


58 ms ± 291 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [28]:
%%timeit -n1 -r1
out = np.sum(vs)

2.12 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [68]:
syms = ""
for i in range(784):
    id = str(uuid.uuid4()).replace("-","")
    syms += 's'+id+str(i)+','

In [69]:
from sympy import symbols

In [70]:
symbols = symbols(syms[:-1])

In [71]:
%%timeit -n1 -r1
out = np.sum(symbols)

2.01 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [2]:
%%timeit -n1 -r1
out = np.sum([[x.value.poly for x in row] for row in data[0].tolist()])

2.05 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [3]:
# %%timeit -n1 -r1
# result = data[0].sum()

In [4]:
%%timeit -n1 -r1
result = data[1].sum()

2.06 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [5]:
%%timeit -n1 -r1
result = data[0].sum()

2.07 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [6]:
%%timeit -n1 -r1
result = data[1].sum()

2.06 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [5]:
acc = AdversarialAccountant(max_budget=7000)

In [6]:
out = result.publish(acc=acc,sigma=1.5)
acc.print_ledger()

KeyboardInterrupt: 

In [48]:
acc.print_ledger()

<Entity:Constance_Weddle>	2.365443354618569


In [6]:
=

x = Tensor(np.array([[1,1],[1,0],[0,1],[0,0]])).private(min_val=0, max_val=1, entities=entities, is_discrete=True)
y = Tensor(np.array([[1],[1],[0],[0]])).private(min_val=0, max_val=1, entities=entities, is_discrete=False)

_weights = Tensor(np.random.uniform(size=(2,1)))

SyntaxError: invalid syntax (<ipython-input-6-07842543fea2>, line 1)

In [2]:
weights = _weights + 0
# acc = AdversarialAccountant(max_budget=7)  # causes error at end of budget


for i in range(3):
    batch_loss = 0

    pred = x.dot(weights)
    pre_loss = np.square(y-pred)
    loss = np.mean(pre_loss, axis=1)
#     loss = np.mean(pre_loss)    
    ledger = loss.backward(accumulate_grads=True)
    weight_grad = (weights.grad*0.3)
    weight_grad_noise = weight_grad.publish(acc=acc, sigma=0.005)
    weights = weights - weight_grad_noise
    ledger.zero_grads()
    batch_loss += loss.value
    print(np.sum([loss.value]))
    
acc.print_ledger()


0.17169736596498625
0.07441205306476974
0.04001946718047936
<Entity:George>	602.4247883171074
<Entity:Kritika>	602.4247883171074
<Entity:Madhava>	602.4247883171074
<Entity:Tudor>	602.4247883171074


In [13]:
x = PhiScalar(0,0.01,1)

In [14]:
out = x * x

In [15]:
out.publish(acc=acc)

<IntermediatePhiScalar: (0.0 < 0.000100000000000000 < 1.0)>
found one!


[0.703353940208477]

In [10]:
acc.print_ledger()

<Entity:George>	0.0
<Entity:Richard_Alvarez>	0.0002737137892581691


In [1]:
x.entity

NameError: name 'x' is not defined

In [7]:
import sympy as sym

In [9]:
a,b = sym.symbols('a,b')

In [10]:
y = a + b

In [38]:
from sympy.core.numbers import Number
class Float2(Number):
    
    def __init__(self, val):
        self.val = val
        
    def __add__(self, other):
        print("adding")
        return Float(self.val - other.val)

In [39]:
out = y.subs({a:Float2(3)})
out

b + 3

In [40]:
out.subs({b:Float2(2)})

5

In [9]:
# loss[0].value.poly

IndexError: too many indices for array

24ffce1fe8ab44a583abecf0b2e1ca69_c06e7e291fdb4cb6b5ec0b9abbabf1bf + 3*3175e5ebc4944304bb4a1a010fa5003d_162afbde9a87417a9a49c14568ea1c11 + 4*5ec1e3788c314340a6ea45342375281e_162afbde9a87417a9a49c14568ea1c11 + 6*68ed3172c1f6417faa4f5ca24611e12d_d59083b02ef74bdf8122e0c51454aa71 + 6*98d310c205b743be9af907ff4646238b_b63effdec6ba47bf93a126d0b5cc7086 + d082ac7660e84d0c806997c2dbe4f600_b63effdec6ba47bf93a126d0b5cc7086 + 5*eeffa282abe44326ae5fc1b8a6f8c6a0_d59083b02ef74bdf8122e0c51454aa71 + 2*fdefb3e070394d5cbf22820ac0ab7b00_c06e7e291fdb4cb6b5ec0b9abbabf1bf

In [ ]:
## first-pass methods to try to get into AutogradTensor

ops = ['T',
 '__abs__',
#  '__add__',
#  '__and__',
#  '__array__',
#  '__array_finalize__',
#  '__array_function__',
#  '__array_interface__',
#  '__array_prepare__',
#  '__array_priority__',
#  '__array_struct__',
#  '__array_ufunc__',
#  '__array_wrap__',
#  '__bool__',
#  '__class__',
#  '__complex__',
#  '__contains__',
#  '__copy__',
#  '__deepcopy__',
#  '__delattr__',
#  '__delitem__',
#  '__dir__',
 '__divmod__',
#  '__doc__',
 '__eq__',
#  '__float__',
 '__floordiv__',
#  '__format__',
 '__ge__',
#  '__getattribute__',
 '__getitem__',
 '__gt__',
#  '__hash__',
#  '__iadd__',
#  '__iand__', # trigger an error for inline 
#  '__ifloordiv__',
#  '__ilshift__',
#  '__imatmul__',
#  '__imod__',
#  '__imul__',
 '__index__',
#  '__init__',
#  '__init_subclass__',
#  '__int__',
 '__invert__',
#  '__ior__',
#  '__ipow__',
#  '__irshift__',
#  '__isub__',
 '__iter__',
#  '__itruediv__',
#  '__ixor__',
 '__le__',
 '__len__',
 '__lshift__',
 '__lt__',
 '__matmul__',
#  '__mod__',
 '__mul__',
 '__ne__',
 '__neg__',
#  '__new__',
#  '__or__',
#  '__pos__',
 '__pow__',
 '__radd__',
#  '__rand__',
#  '__rdivmod__',
#  '__reduce__',
#  '__reduce_ex__',
 '__repr__',
 '__rfloordiv__',
 '__rlshift__',
 '__rmatmul__',
#  '__rmod__',
 '__rmul__',
#  '__ror__',
 '__rpow__',
 '__rrshift__',
 '__rshift__',
 '__rsub__',
 '__rtruediv__',
#  '__rxor__',
#  '__setattr__',
#  '__setitem__',
#  '__setstate__',
 '__sizeof__',
 '__str__',
 '__sub__',
#  '__subclasshook__',
 '__truediv__',
#  '__xor__',
#  'all',
#  'any',
 'argmax',
 'argmin',
#  'argpartition',
 'argsort',
#  'astype',
#  'base',
#  'byteswap',
 'choose',
 'clip',
#  'compress',
#  'conj',
#  'conjugate',
 'copy',
#  'ctypes',
 'cumprod',
 'cumsum',
#  'data',
 'diagonal',
 'dot',
#  'dtype',
#  'dump',
#  'dumps',
#  'fill',
#  'flags',
 'flat',
 'flatten',
#  'getfield',
#  'imag',
 'item',
 'itemset',
 'itemsize',
 'max',
 'mean',
 'min',
#  'nbytes',
 'ndim',
#  'newbyteorder',
#  'nonzero',
#  'partition',
 'prod',
#  'ptp',
#  'put',
#  'ravel',
#  'real',
 'repeat',
 'reshape',
 'resize',
#  'round',
#  'searchsorted',
#  'setfield',
#  'setflags',
#  'shape',
#  'size',
 'sort',
 'squeeze',
 'std',
#  'strides',
 'sum',
 'swapaxes',
 'take',
#  'tobytes',
#  'tofile',
#  'tolist',
#  'tostring',
#  'trace',
 'transpose',
#  'var',
#  'view'
      ]